# Beautiful Soup (BS4)

BS4 es una librería para extraer datos de documentos con lenguajes de marcado como HTML o XML. Permite navegar por el árbol del documento, estraer datos, modificarlo, etc. Como veremos, es recomendable conocer el funcionamiento de las reglas CSS para sacar el máximo partido a esta librería.

Como vemos en este primer ejemplo, BS4 tan solo analiza texto en formato HTML o XML. Aquí está la [documentación completa de Beautiful Soup 4.](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [1]:
from bs4 import BeautifulSoup

html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

# Cargamos el 
soup = BeautifulSoup(html_doc, 'lxml')

Podemos navegar directamente por la estructura usando `.`


In [2]:
soup.title


<title>The Dormouse's story</title>

In [3]:
soup.p


<p class="title"><b>The Dormouse's story</b></p>

In [4]:
soup.a.text

'Elsie'

Podemos acceder también a los atributos de un elemento

In [5]:
soup.a.attrs

{'class': ['sister'], 'href': 'http://example.com/elsie', 'id': 'link1'}

In [6]:
soup.a['class']
# Equivalente a soup.a.attrs['class]

['sister']

También podemos acceder a padre, hijos y hermanos de un elemento

In [7]:
soup.a.nextSibling

',\n'

In [8]:
soup.a.findNextSibling('a')

<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>

In [9]:
soup.a.parent.name

'p'

Realmente, la parte más interesante es la de encontrar todos los elementos de un determinado tipo, o que sean de una determinada clase, o en el que alguno de sus atributos tenga algún valor especial. Esto lo haremos con:

- `find`: encuentra el primer elemento
- `findall`: genera una lista con todos los elementos que cumplen la condición.

In [10]:
soup.findAll('a')

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [11]:
soup.find('a', attrs={"class": "sister"})

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [12]:
# Devuelve None en caso de no existir
print(soup.find('a', attrs={"class": "brother"}))

None


De esta forma, ya podemos extraer todos los enlaces de un documento en HTML

In [13]:
links = soup.findAll('a')
for l in links:
    print(l['href'])

http://example.com/elsie
http://example.com/lacie
http://example.com/tillie


En realidad, existe una forma aún más interesante de realizar esto y es utilizando la función `select` (o `select_one`, si solo nos queremos quedar con el primero, que recibe como parámetro un selector CSS para decidir con qué elementos me quiero quedar 

In [14]:
# Ignoremos el párrafo de título
stories = soup.select('p.story')
for t in stories:
    print (t.text)

Once upon a time there were three little sisters; and their names were
Elsie,
Lacie and
Tillie;
and they lived at the bottom of a well.
...


In [15]:
# Ignoremos el párrafo de título
stories = soup.select('p.story > a')
for t in stories:
    print (t['id'])

link1
link2
link3


# BS4 y request

Para el objetivo de este taller, lo más interesante es combinar la descarga de una web con el uso de BS4. Para ello, proponemos el uso de la librería `request` de la siguiente manera:

In [16]:
from bs4 import BeautifulSoup
import requests

def procesarPagina(url):
    """
    Carga y  procesa el contenido de una URL usando request
    Muestra un mensaje de error en caso de no poder cargar la página
    """
     # Realizamos la petición a la web
    req = requests.get(url)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:

        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text,"lxml")
        
        # Procesamos el HTML descargado
        return procesaHTML(html,url)        
        
    else:
        print ("ERROR {}".format(statusCode))

def procesaHTML(html, url=""):
    """
    Procesa el contenido HTML de una página web
    html es un objeto BS4
    url es la URL de la página contenida en html_doc
    """
    # Decide aquí lo que quieres hacer con el contenido 
    return

## Ejemplo: Books to scrape

Vamos a probar cómo hacer scraping con BS4 utilizando el sitio web [Books to scrape](http://books.toscrape.com). Como su nombre indica, es un _sandbox_ donde nos dejan  hacer pruebas de web scraping. Utilizando las herramientas de desarrollador del navegador podemos analizar cuál es la estructura de la web que queremos analizar.

Vamos primeramente a entender cómo es la página de un producto para luego ver cómo extraeríamos el catálogo completo.

### Organización de cada producto

La estructura más importante de un producto es la que aquí aparece:

```
article.product_page
    div.product_main
        h1 
            Texto = Título
        p.price_color
            Texto = Precio
        p.star-rating
            La otra clase representa la valoración (One, Two, Three, Four, Five)
    div#product_description
        sibling p --> descripción
    
    table
        Cada fila tiene info adicional:
        UPC
        Product type
        Price (excl. tax)
        ...
        
```

Por ejemplo, comencemos obteniendo el título del producto

In [17]:
def procesaHTML(html, url=""):
    titulo = html.select_one(".product_main h1").text
    print ("Título:"+ titulo)

procesarPagina("http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html")


Título:A Light in the Attic


Vamos a ir creando un objeto con toda la información de la página

In [18]:
# Nombres de las clases que representan las valoraciones de un producto
star = ["One", "Two", "Three", "Four", "Five"]

def starToInt (rating):
    """
    Convierte un rating en forma textual a un rating numérico
    Devuelve el número equivalente, o 0, si el rating no es válido
    """
    try:
        return star.index(rating) + 1
    except:
        return 0

def procesaHTML(html, url=""):
    libro = {}
    
    prodMain = html.select_one(".product_main")

    # Título
    titulo = prodMain.select_one("h1").text
    libro['titulo'] = titulo

    # Precio (eliminamos los caracteres anteriores que representan las libras)
    precio = prodMain.select_one("p.price_color").text
    libro['precio'] = precio[2:]
    
    # Valoración
    # 1. Obtenemos las clases
    ratingClasses = prodMain.select_one("p.star-rating")["class"]
    
    # 2. Nos quedamos con la intersección
    ratingText = list(set(ratingClasses).intersection(set(star)))
    
    # 3. Lo convertimos a un valor numérico
    if (len(ratingText)==1):
        libro['valoracion'] = starToInt(ratingText[0])
    else:
        libro['valoracion'] = 0
        

    return libro

    
procesarPagina("http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html")


{'precio': '51.77', 'titulo': 'A Light in the Attic', 'valoracion': 3}

Procesar la descripción nos hace buscar el hermano de un elemento:


In [19]:
def procesaHTML(html, url=""):
    libro = {}
    
    prodMain = html.select_one(".product_main")

    # Título
    titulo = prodMain.select_one("h1").text
    libro['titulo'] = titulo

    # Precio (eliminamos los caracteres anteriores que representan las libras)
    precio = prodMain.select_one("p.price_color").text
    libro['precio'] = precio[2:]
    
    # Valoración
    # 1. Obtenemos las clases
    ratingClasses = prodMain.select_one("p.star-rating")["class"]
    
    # 2. Nos quedamos con la intersección
    ratingText = list(set(ratingClasses).intersection(set(star)))
    
    # 3. Lo convertimos a un valor numérico
    if (len(ratingText)==1):
        libro['valoracion'] = starToInt(ratingText[0])
    else:
        libro['valoracion'] = 0
        
    # Descripción del producto
    # 1. Buscamos el elemento que hace de título
    prodDescription = html.find(id="product_description")
    
    # 2. Buscamos el siguiente hermano con etiqueta p
    if prodDescription is None:
        libro['descripcion'] = ""
    else:
        libro['descripcion'] = prodDescription.find_next_sibling('p').text
    
    return libro

    
procesarPagina("http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html")

{'descripcion': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby, I think someone down here'sGot it in for you. Shel, 

### Organización del catálogo

Una vez que sabemos cómo procesar un producto vamos a intentar obtener el catálogo completo. La estructura de cada uno de los productos de la página de un catálogo es esta:

```
article.product_pod
    img[src] = URL imagen
    h3
        Texto = Título del libro
        a[href] = enlace
``` 

Se muestran 20 por página y hay un botón `Next` para avanzar. Sin embargo, si nos fijamos un poco veremos que cada vez que pulsamos en el botón nos lleva a una nueva página con una URL del tipo `http://books.toscrape.com/catalogue/page-X.html`. Además, podemos ver que lanza un error 404 si llegamos al final del catálogo.

De acuerdo a esto, vamos a ver cómo obtener los enlaces de cada uno de los productos y cómo crear una lista de libros:


In [20]:
def procesaPaginaCatalogo(url):
     # Realizamos la petición a la web
    req = requests.get(url)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:

        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text,"lxml")
        
        # Procesamos el HTML descargado
        products = html.select('article.product_pod')
        for prod in products:
            enlace = prod.select_one('h3 > a')
            print(enlace['href'])
            
procesaPaginaCatalogo("http://books.toscrape.com/catalogue/page-1.html")
    

a-light-in-the-attic_1000/index.html
tipping-the-velvet_999/index.html
soumission_998/index.html
sharp-objects_997/index.html
sapiens-a-brief-history-of-humankind_996/index.html
the-requiem-red_995/index.html
the-dirty-little-secrets-of-getting-your-dream-job_994/index.html
the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html
the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html
the-black-maria_991/index.html
starving-hearts-triangular-trade-trilogy-1_990/index.html
shakespeares-sonnets_989/index.html
set-me-free_988/index.html
scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html
rip-it-up-and-start-again_986/index.html
our-band-could-be-your-life-scenes-from-the-american-indie-underground-1981-1991_985/index.html
olio_984/index.html
mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html
libertarianism-for-beginners_982/index.html
its-only-the-himalayas_981/

Los enlaces son relativos por lo que necesitamos definir la ruta completa. Si desde el navegador accedemos a uno de los productos vemos que el prefijo que usa es `http://books.toscrape.com/catalogue/`. Lo añadimos y procesamos cada enlace. Luego vamos componiendo una lista de productos con todos ellos.

In [21]:
def procesaPaginaCatalogo(url, prefix, productList):
     # Realizamos la petición a la web
    req = requests.get(url)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:

        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text,"lxml")
        
        # Procesamos el HTML descargado
        products = html.select('article.product_pod')
        for prod in products:
            enlace = prod.select_one('h3 > a')
            productList.append(procesarPagina(prefix+enlace['href']))
            
listaProductos = []
procesaPaginaCatalogo("http://books.toscrape.com/catalogue/page-1.html", "http://books.toscrape.com/catalogue/", listaProductos)
len(listaProductos)

20

Solo queda iterar por las páginas de los catálogos y parar si tenemos un error 404

In [22]:
def procesaPaginaCatalogo(url, prefix, productList):
    """
    Devuelve True si hemos llegado al final del catálogo, False en otro caso
    """
     # Realizamos la petición a la web
    req = requests.get(url)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:

        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text,"lxml")
        
        # Procesamos el HTML descargado
        products = html.select('article.product_pod')
        for prod in products:
            enlace = prod.select_one('h3 > a')
            producto = procesarPagina(prefix+enlace['href'])
            producto['enlace'] = prefix+enlace['href']
            productList.append(producto)
        return False
    
    if statusCode == 404:
        return True
        
listaProductos = []
# Probamos solo con las dos últimas páginas
i=48
while ( not procesaPaginaCatalogo("http://books.toscrape.com/catalogue/page-{}.html".format(i), "http://books.toscrape.com/catalogue/", listaProductos)):
    i=i+1

len(listaProductos)

60

Finalmente cargaremos todos los datos en un dataframe de pandas para procesarlo, extraer información y guardarlo a un CSV


In [23]:
import pandas as pd
df = pd.DataFrame(listaProductos)
df.to_csv("listaProductos.csv", sep=";", index=False)

## Extra: Municipios y habitantes de Madrid

Para terminar y a modo de ejemplo extra, extraeremos los municipios y número de habitantes de la Comunidad de Madrid de la Wikipedia

In [24]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = "https://es.wikipedia.org/wiki/Anexo:Municipios_de_la_Comunidad_de_Madrid"

# Realizamos la petición a la web
req = requests.get(url)

# Comprobamos que la petición nos devuelve un Status Code = 200
statusCode = req.status_code
if statusCode == 200:

    # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
    html = BeautifulSoup(req.text, 'html.parser')
    

    # Obtenemos cada una de las filas de la tabla
    content = []
    rows = html.find_all('tr')
    for r in rows:
        celdas=r.find_all('td')
        # Nos quedamos con el contenido
        if len(celdas)>0:
            # En lugar de un separador de miles, se ha usado un caracter parecido a un espacio en blanco
            # por lo que en la celda de habitantes hay que eliminar todos los caracteres que no sean números
            content.append([celdas[0].string, ''.join(c for c in celdas[1].string if c.isdigit())])
        

else:
    print (statusCode)

df = pd.DataFrame(content, columns= ['Nombre', 'Habitantes'])
df

,Nombre,Habitantes
0,La Acebeda,66
1,Ajalvir,4455
2,Alameda del Valle,199
3,El Álamo,9149
4,Alcalá de Henares,194310
5,Alcobendas,114864
6,Alcorcón,168141
7,Aldea del Fresno,2616
8,Algete,20419
9,Alpedrete,14240
